In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials #To access authorised Spotify data

from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import time
from requests import get
import os
import datetime
from io import StringIO

In [2]:
client_id = '##############################'
client_secret = '##############################'

In [3]:
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) #spotify object to access API

In [4]:
start_date = datetime.date(2019,1,1)
end_date = datetime.date(2020,6,1)

In [5]:
BASE_URL='https://spotifycharts.com'

# Extracting Top 200 Songs from Jan 2019 to Jun 2020

In [ ]:
final_df = pd.DataFrame()
while start_date<end_date:
    print(f'Reading file for {start_date}')
    url = 'https://spotifycharts.com/regional/us/daily/%s' %start_date.strftime('%Y-%m-%d')
    get_page = get(url)#, headers = hdr)
    get_soup = BeautifulSoup(get_page.content, 'html.parser')
    get_url = BASE_URL+get_soup.find('div').find('a',attrs={'class':'header-csv'}).attrs['href']
    get_file = get(get_url)
    fileobj = StringIO(get_file.text)
    df = pd.read_csv(fileobj, header=1)
    df['date'] = start_date
    df['region'] = 'USA'
    df['track_id'] = df['URL'].str[-22:]
    final_df = final_df.append(df)
    final_df.to_csv('spotify_2019_2020_top200.csv', index=False)
    start_date += datetime.timedelta(1)

# Extracting Track Feature Information

In [ ]:
%%time
track_features = pd.DataFrame()
for keys, tracks in enumerate(final_df.track_id.unique()):
    print(f'Reading features for track - {keys}')
    tf = sp.audio_features(tracks)
    tf = pd.DataFrame(tf)
    tf['track_id']=tracks
    track_features = track_features.append(tf)
    track_features.to_csv('track_features.csv', index=False)

# Extracting Artist Information

In [ ]:
%%time
artist_features = pd.DataFrame()
for keys, name in enumerate(final_df.Artist.unique()):
    print(f'Reading features for artist - {keys} - {name}')
    results = sp.search(q='artist:' + name, type='artist')
    #assume first search result is the artist we want
    try:
        if results['artists']['items'][0]['name'] == name:
            artist = {}
            artist['artist'] = results['artists']['items'][0]['name']
            artist['genres'] = results['artists']['items'][0]['genres']
            artist['popularity'] = results['artists']['items'][0]['popularity']
            artist['followers'] = results['artists']['items'][0]['followers']['total']
            artist['artist_id'] = results['artists']['items'][0]['uri'][-22:]
            artist = pd.DataFrame.from_dict(artist)
            artist_features = artist_features.append(artist)
            artist_features.to_csv('artist_features.csv', index=False)
    except:
        print(f'No features for artist - {name}')
        artist_features = artist_features.append(artist)
        artist_features.to_csv('artist_features.csv', index=False)